In [1]:
!pip install reverse_geocoder

In [2]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import folium

from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster
from folium.plugins import HeatMapWithTime

from bs4 import BeautifulSoup

import reverse_geocoder as rg 
import pprint 
import pickle


In [3]:
# %load src/merging geospatial info.py


import pandas as pd
import pickle


rg = pd.read_pickle('data/rg.pkl')

get_ipython().run_line_magic('run', 'src/pipeline')
D = FraudData('data/data.json')
df = D.get_enriched_df()

loc = df[['venue_latitude','venue_longitude','object_id']]
loc = loc[~loc['venue_latitude'].isnull()]

loc2 = loc.join(rg)
loc3 = loc2.drop(['venue_latitude', 'venue_longitude'],axis = 1)
df_geo = df.merge(loc3, how='left', right_on = 'object_id', left_on = 'object_id')





In [4]:
df = df_geo.copy()

#### Geographical visualization of all the events

In [5]:
default_loc = df['venue_latitude'].mean(), df['venue_longitude'].mean()

def generateBaseMap(default_location=default_loc, default_zoom_start= 3):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map




In [6]:
df_copy = df.copy()
df_copy['count'] = 1
base_map = generateBaseMap()
HeatMap(data=df_copy[['venue_latitude', 'venue_longitude', 'count']].groupby(['venue_latitude', 'venue_longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)


base_map.save('images/events_heatmap.html')

In [7]:
df2 = df_copy[~df_copy['venue_latitude'].isnull()]
df2.shape

(13261, 65)

In [8]:
lat = df2['venue_latitude'].tolist()
lon = df2['venue_longitude'].tolist()
locations = list(zip(lat, lon))

map1 = folium.Map(location=default_loc, zoom_start=3)
FastMarkerCluster(data=locations).add_to(map1)
map1.save('images/events_ClusteredMarker.html')

#### Geographical visualization of fraudulent events only

In [9]:
fraud_all = df.query('label == 1')
fraud_all.shape

(1293, 64)

In [10]:
fraud = df2.query('label == 1')
fraud.shape

# 70% of the fraudulent events have geo info.

(923, 65)

In [11]:
fraud['count'] = 1
base_map = generateBaseMap()
HeatMap(data=fraud[['venue_latitude', 'venue_longitude', 'count']].groupby(['venue_latitude', 'venue_longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)
base_map.save('images/fraud_heatmap.html')

/Users/karensheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
lat = fraud['venue_latitude'].tolist()
lon = fraud['venue_longitude'].tolist()
locations = list(zip(lat, lon))

map2 = folium.Map(location= default_loc, zoom_start= 3)
FastMarkerCluster(data=locations).add_to(map2)
map2.save('images/fraud_ClusteredMarker.html')

#### Geographical Visualization with Popup

In [13]:
# default_loc = df['venue_latitude'].mean(), df['venue_longitude'].mean()

# def plot_station_counts(df):
#     # generate a new map
#     folium_map = folium.Map(location= default_loc,
#                             zoom_start=13)
                           

#     # for each row in the data, add a cicle marker
#     for index, row in df.iterrows():
#         # calculate net departures
# #         net_departures = (row["Departure Count"]-row["Arrival Count"])
        
#         # generate the popup message that is shown on click.
#         popup_text = "event name: {}<br> venue: {}<br> curreny: {}"
#         popup_text = popup_text.format(row["name"],
#                           row["venue_address"],
#                           row["currency"])
        
#         # radius of circles
#         radius = row['num_payouts']
        
#         # choose the color of the marker
#         if row['label'] == 1:
#             # color="#FFCE00" # orange
#             # color="#007849" # green
#             color="#E37222" # tangerine
#         else:
#             # color="#0375B4" # blue
#             # color="#FFCE00" # yellow            
#             color="#0A8A9F" # teal
        
#         # add marker to the map
#         folium.CircleMarker(location=(row["venue_latitude"],
#                                       row["venue_longitude"]),
#                             radius=radius,
#                             color=color,
#                             popup=popup_text,
#                             fill=True).add_to(folium_map)
#     return folium_map

In [14]:
# map_t = plot_station_counts(fraud)

# map_t.save('images/fraud_map_test.html')

In [15]:

# locationlist = fraud[["venue_latitude","venue_longitude"]].values.tolist()
# labels = fraud["name"].values.tolist()

# # Create map and drop points onto it
# m = folium.Map(location= default_loc, zoom_start=5)
# for point in range(len(locationlist)):
#     popup = folium.Popup(labels[point], parse_html=True)
#     folium.Marker(locationlist[point], popup=popup).add_to(m)



In [16]:
# m.save('images/fraud_ClusteredMarker_popup.html')

In [18]:
fraud.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 923 entries, 0 to 14336
Data columns (total 65 columns):
acct_type             923 non-null object
approx_payout_date    923 non-null int64
body_length           923 non-null int64
channels              923 non-null int64
country               923 non-null object
currency              923 non-null object
delivery_method       911 non-null float64
description           923 non-null object
email_domain          923 non-null object
event_created         923 non-null int64
event_end             923 non-null int64
event_published       901 non-null float64
event_start           923 non-null int64
fb_published          923 non-null int64
gts                   923 non-null float64
has_analytics         923 non-null int64
has_header            391 non-null float64
has_logo              923 non-null int64
listed                923 non-null object
name                  923 non-null object
name_length           923 non-null int64
num_order        

#### Heatmap with Time

In [26]:
fraud['ec_mth_yr'] = pd.to_datetime(fraud.ec_year.astype(str) + '-' + fraud.ec_month.astype(str))

/Users/karensheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
fraud.ec_mth_yr.value_counts()

2013-09-01    68
2012-10-01    52
2012-03-01    51
2012-11-01    50
2012-12-01    44
2013-08-01    42
2013-01-01    40
2013-07-01    40
2012-04-01    39
2013-06-01    36
2012-05-01    35
2012-06-01    33
2012-09-01    29
2012-07-01    27
2012-02-01    26
2013-03-01    25
2013-02-01    22
2011-12-01    20
2013-04-01    19
2011-02-01    18
2011-01-01    18
2011-05-01    17
2011-04-01    17
2011-07-01    17
2012-08-01    17
2011-08-01    16
2013-05-01    14
2013-10-01    13
2012-01-01    13
2011-03-01    11
2011-06-01    11
2011-09-01     6
2011-11-01     6
2008-03-01     6
2011-10-01     5
2007-02-01     3
2009-03-01     3
2010-02-01     3
2009-11-01     3
2010-05-01     1
2010-12-01     1
2010-11-01     1
2010-08-01     1
2010-07-01     1
2009-10-01     1
2010-01-01     1
2010-09-01     1
Name: ec_mth_yr, dtype: int64

In [31]:
event_dt_list = []
for dt in fraud.ec_mth_yr.sort_values().unique():
    event_dt_list.append(fraud.loc[fraud.ec_mth_yr == dt, ['venue_latitude', 'venue_longitude', 'count']].groupby(['venue_latitude', 'venue_longitude']).sum().reset_index().values.tolist())

In [32]:
base_map = generateBaseMap()
HeatMapWithTime(event_dt_list, radius=3, gradient={0.8: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map)
base_map.save('images/fraud_heatmap_time.html')



In [33]:
base_map